In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [2]:
# Misalnya df_1 dan df_2 sudah di-load
df_1 = pd.read_csv('../data/data_1.csv', dtype={5: str, 7: str})
df_2 = pd.read_csv('../data/data_2.csv', dtype={5: str, 7: str})

# 1. Ambil semua data dengan attack == 0
df_0_all = pd.concat([
    df_1[df_1['attack'] == 0],
    df_2[df_2['attack'] == 0]
])

# 2. Gabungkan semua data dengan attack == 1 dari kedua DataFrame
df_1_all = pd.concat([
    df_1[df_1['attack'] == 1],
    df_2[df_2['attack'] == 1]
])

# 3. Ambil 20.000 sample secara acak dari attack == 1
df_1_sampled = df_1_all.sample(n=20000, random_state=42)

# 4. Gabungkan kembali kedua data
df_balanced = pd.concat([df_0_all, df_1_sampled])

# (Opsional) Reset index
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Cek distribusi akhir
print(df_balanced['attack'].value_counts())

attack
1    20000
0     6934
Name: count, dtype: int64


In [3]:
print(df_balanced.columns.tolist())

['pkSeqID', 'stime', 'flgs', 'proto', 'saddr', 'sport', 'daddr', 'dport', 'pkts', 'bytes', 'state', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'smac', 'dmac', 'sum', 'min', 'max', 'soui', 'doui', 'sco', 'dco', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'attack', 'category', 'subcategory ']


In [4]:
# Hapus spasi di awal/akhir nama kolom
df_balanced.columns = df_balanced.columns.str.strip()

In [ ]:
# Berdasarkan paper yang membahas dataset Bot-IoT, beberapa kolom tidak relevan untuk analisis lebih lanjut.
# Kolom-kolom dihapus dikarenakan dianggap dapat menyebabkan model oferfitting.
columns_to_drop = [
    'pkSeqID', 'saddr', 'daddr', 'ltime', 'stime', 'seq', 'category', 'subcategory'
]
df_balanced = df_balanced.drop(columns=columns_to_drop, errors='ignore')

In [6]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26934 entries, 0 to 26933
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   flgs    26934 non-null  object 
 1   proto   26934 non-null  object 
 2   sport   26534 non-null  object 
 3   dport   26534 non-null  object 
 4   pkts    26934 non-null  int64  
 5   bytes   26934 non-null  int64  
 6   state   26934 non-null  object 
 7   dur     26934 non-null  float64
 8   mean    26934 non-null  float64
 9   stddev  26934 non-null  float64
 10  smac    0 non-null      float64
 11  dmac    0 non-null      float64
 12  sum     26934 non-null  float64
 13  min     26934 non-null  float64
 14  max     26934 non-null  float64
 15  soui    0 non-null      float64
 16  doui    0 non-null      float64
 17  sco     0 non-null      float64
 18  dco     0 non-null      float64
 19  spkts   26934 non-null  int64  
 20  dpkts   26934 non-null  int64  
 21  sbytes  26934 non-null  int64  
 22

In [7]:
df_balanced.head()

,flgs,proto,sport,dport,pkts,bytes,state,dur,mean,stddev,...,sco,dco,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack
0,e,udp,52841,53,2,176,INT,5.047946,0.000000,0.0,...,NaN,NaN,2,0,176,0,0.198100,0.19810,0.00000,0
1,e,tcp,60134,6788,2,120,RST,0.000114,0.000114,0.0,...,NaN,NaN,1,1,60,60,8771.929688,0.00000,0.00000,1
2,e,tcp,56304,15224,2,120,RST,0.033741,0.033741,0.0,...,NaN,NaN,1,1,60,60,29.637531,0.00000,0.00000,1
3,e,tcp,59460,25280,2,120,RST,0.032499,0.032499,0.0,...,NaN,NaN,1,1,60,60,30.770178,0.00000,0.00000,1
4,e,tcp,54892,1883,6,662,RST,0.000244,0.000244,0.0,...,NaN,NaN,3,3,456,206,20491.802734,8196.72168,8196.72168,1


In [8]:
# Menghapus kolom yang tidak relevan atau yang seluruhnya berisi nilai null.
# Kolom-kolom ini ('smac', 'dmac', 'soui', 'doui', 'sco', 'dco') tidak memberikan informasi yang berguna untuk analisis.
df_balanced.drop(columns=["smac", "dmac", "soui", "doui", "sco", "dco"], inplace=True)

In [9]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26934 entries, 0 to 26933
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   flgs    26934 non-null  object 
 1   proto   26934 non-null  object 
 2   sport   26534 non-null  object 
 3   dport   26534 non-null  object 
 4   pkts    26934 non-null  int64  
 5   bytes   26934 non-null  int64  
 6   state   26934 non-null  object 
 7   dur     26934 non-null  float64
 8   mean    26934 non-null  float64
 9   stddev  26934 non-null  float64
 10  sum     26934 non-null  float64
 11  min     26934 non-null  float64
 12  max     26934 non-null  float64
 13  spkts   26934 non-null  int64  
 14  dpkts   26934 non-null  int64  
 15  sbytes  26934 non-null  int64  
 16  dbytes  26934 non-null  int64  
 17  rate    26934 non-null  float64
 18  srate   26934 non-null  float64
 19  drate   26934 non-null  float64
 20  attack  26934 non-null  int64  
dtypes: float64(9), int64(7), object(5)


In [12]:
df_balanced.head()

,flgs,proto,sport,dport,pkts,bytes,state,dur,mean,stddev,...,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate,attack,subcategory
0,e,udp,52841,53,2,176,INT,5.047946,0.000000,0.0,...,0.000000,2,0,176,0,0.198100,0.19810,0.00000,0,Normal
1,e,tcp,60134,6788,2,120,RST,0.000114,0.000114,0.0,...,0.000114,1,1,60,60,8771.929688,0.00000,0.00000,1,Service_Scan
2,e,tcp,56304,15224,2,120,RST,0.033741,0.033741,0.0,...,0.033741,1,1,60,60,29.637531,0.00000,0.00000,1,Service_Scan
3,e,tcp,59460,25280,2,120,RST,0.032499,0.032499,0.0,...,0.032499,1,1,60,60,30.770178,0.00000,0.00000,1,Service_Scan
4,e,tcp,54892,1883,6,662,RST,0.000244,0.000244,0.0,...,0.000244,3,3,456,206,20491.802734,8196.72168,8196.72168,1,Service_Scan


In [13]:
df_balanced.drop_duplicates(inplace=True)

In [14]:
missing = df_balanced.isnull().sum()
print(missing[missing > 0])

sport    307
dport    307
dtype: int64


In [15]:
# Menghapus baris yang memiliki nilai null di kolom 'sport' atau 'dport'.
# Baris dengan data yang hilang tidak dapat digunakan untuk pembuatan model.
df_balanced.dropna(subset=["sport", "dport"], inplace=True)

In [16]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26265 entries, 0 to 26933
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   flgs         26265 non-null  object 
 1   proto        26265 non-null  object 
 2   sport        26265 non-null  object 
 3   dport        26265 non-null  object 
 4   pkts         26265 non-null  int64  
 5   bytes        26265 non-null  int64  
 6   state        26265 non-null  object 
 7   dur          26265 non-null  float64
 8   mean         26265 non-null  float64
 9   stddev       26265 non-null  float64
 10  sum          26265 non-null  float64
 11  min          26265 non-null  float64
 12  max          26265 non-null  float64
 13  spkts        26265 non-null  int64  
 14  dpkts        26265 non-null  int64  
 15  sbytes       26265 non-null  int64  
 16  dbytes       26265 non-null  int64  
 17  rate         26265 non-null  float64
 18  srate        26265 non-null  float64
 19  drate    

In [7]:
target = "attack"  
X = df.drop(columns=[target])
y = df[target]

In [8]:
print(y.value_counts())

attack
1    997753
0      1760
Name: count, dtype: int64


In [9]:
high_card_cat = ["saddr", "daddr", "sport", "dport"]
for col in high_card_cat:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

In [10]:
low_card_cat = ["proto", "state", "flgs", "category", "subcategory"]

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
ohe_result = ohe.fit_transform(X[low_card_cat])
ohe_cols = ohe.get_feature_names_out(low_card_cat)

X_ohe = pd.DataFrame(ohe_result, columns=ohe_cols, index=X.index)

In [11]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Buang high-cardinality categorical dari numerik
num_cols = [col for col in num_cols if col not in high_card_cat]

scaler = StandardScaler()
scaled_result = scaler.fit_transform(X[num_cols])

X_scaled = pd.DataFrame(scaled_result, columns=num_cols, index=X.index)

In [13]:
X_processed = pd.concat([X_scaled, X[high_card_cat], X_ohe], axis=1)

In [14]:
final_df = pd.concat([X_processed, y], axis=1)

In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999513 entries, 1 to 999999
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   pkSeqID                   999513 non-null  float64
 1   stime                     999513 non-null  float64
 2   pkts                      999513 non-null  float64
 3   bytes                     999513 non-null  float64
 4   ltime                     999513 non-null  float64
 5   seq                       999513 non-null  float64
 6   dur                       999513 non-null  float64
 7   mean                      999513 non-null  float64
 8   stddev                    999513 non-null  float64
 9   sum                       999513 non-null  float64
 10  min                       999513 non-null  float64
 11  max                       999513 non-null  float64
 12  spkts                     999513 non-null  float64
 13  dpkts                     999513 non-null  float6

In [16]:
missing = final_df.isnull().sum()
print(missing[missing > 0])

Series([], dtype: int64)


In [17]:
final_df.head()

,pkSeqID,stime,pkts,bytes,ltime,seq,dur,mean,stddev,sum,...,flgs_e dD,flgs_e g,flgs_e r,flgs_e s,flgs_eU,category_Normal,category_Reconnaissance,subcategory_Normal,subcategory_Service_Scan,attack
1,-1.732297,-2.482046,0.004015,-0.009365,-2.474600,-1.148851,38.399038,-0.172964,-0.040363,-0.015883,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
2,-1.732294,-2.482026,-0.012164,-0.010411,-2.482075,-1.148839,-0.041588,0.065206,-0.040468,-0.013431,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
4,-1.732287,-2.481636,-0.008119,-0.009470,-2.478748,-1.148804,15.026018,0.310260,1.010213,-0.005917,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
7,-1.732276,-2.481514,-0.012164,-0.010427,-2.481550,-1.148757,0.023230,-0.173102,-0.040468,-0.015890,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
8,-1.732273,-2.481504,-0.012164,-0.010427,-2.481540,-1.148488,0.023257,-0.173102,-0.040468,-0.015890,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0


In [20]:
print(final_df['saddr'].value_counts())

saddr
0     262580
1     250513
2     230126
3     226163
5      17170
8       5088
11      3948
10      3672
7         79
4         42
6         37
19        16
12        13
15        11
13        10
17        10
14        10
18         9
16         9
9          7
Name: count, dtype: int64


In [21]:
print(final_df['daddr'].value_counts())

daddr
26     882445
29      34848
32      27141
31      24595
21       7392
        ...  
74          1
0           1
128         1
140         1
143         1
Name: count, Length: 171, dtype: int64


In [22]:
print(final_df['sport'].value_counts())

sport
32670    65542
35423    65536
35582    65536
34559    64093
37780    63481
         ...  
2311         1
3893         1
2309         1
3891         1
30815        1
Name: count, Length: 47122, dtype: int64


In [23]:
print(final_df['dport'].value_counts())

dport
17632     12675
1058       6180
17574      6059
1059       5666
44806      4478
          ...  
69962         1
18737         1
112388        1
25850         1
65545         1
Name: count, Length: 114001, dtype: int64


In [24]:
final_df.to_csv("../Data/Bot_IoT_processed.csv", index=False)

In [25]:
print("Preprocessing selesai. File disimpan sebagai Bot_IoT_processed.csv")
print("Shape sebelum:", df.shape)
print("Shape sesudah:", final_df.shape)

Preprocessing selesai. File disimpan sebagai Bot_IoT_processed.csv
Shape sebelum: (999513, 29)
Shape sesudah: (999513, 53)


In [26]:
import joblib

# simpan scaler ke file
joblib.dump(scaler, "../Data/scaler.pkl")

['../Data/scaler.pkl']